In [26]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
print("Setup Complete")
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets
import xgboost as xgb

Setup Complete


In [27]:
def classificate(value):
    target='neutral'
    if(value>=0.001 and value<=0.002):
        target='buy'
    elif(value>=0.002):
        target='strong buy'
    elif(value<=-0.001 and value>=-0.002):
        target='sell'
    elif(value<-0.002):
        target='strong sell'
    return target


def classificateWithNumbers(value):
    target=0
    if(value>=0.001 and value<=0.002):
        target=1
    elif(value>=0.002):
        target=2
    elif(value<=-0.001 and value>=-0.002):
        target=-1
    elif(value<-0.002):
        target=-2
    return target

def classificateWithNumbersThreeTarget(value):
    target=0
    if(value>=0.0015):
        target=1
    elif(value<=-0.0015):
        target=-1
    else:
        target=0
    return target



def classificateWithIncreasingNumbers(value):
    target=3
    if(value>=0.0015 and value<=0.0025):
        target=4
    elif(value>=0.0025):
        target=5
    elif(value<=-0.0015 and value>=-0.0025):
        target=2
    elif(value<-0.0025):
        target=1
    return target

def correlation_heatmap(df):
    correlations = df.corr()

    fig, ax = plt.subplots(figsize=(40,40))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.1, annot=True, cbar_kws={"shrink": .7})
    sns.set(font_scale=2)
    plt.show();

def aboveValue(openList,inspectList,df,name):
    results=[]
    for i in range(len(inspectList)):
        if (openList[i]>inspectList[i]):
            results.append(1)
        else:
            results.append(0)
    df[name]=results
    

In [28]:
data1 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_06', sep=",", header=None)
data1.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data2 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_07', sep=",", header=None)
data2.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data3 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_08', sep=",", header=None)
data3.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data4 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_09', sep=",", header=None)
data4.columns = ["time", "open", "high", "low","close","volume"]

data= pd.concat([data1,data2,data3,data4], axis=0)
data

#data=data4

,time,open,high,low,close,volume,drop1,drop2
0,1505706180,3761.68,3775.45,3761.66,3775.32,1.281182,4832.356020,3771.796449
1,1505706240,3769.29,3769.84,3768.89,3769.84,1.352133,5096.716753,3769.390135
2,1505706300,3772.93,3773.01,3772.93,3773.01,0.196074,739.784612,3772.993719
3,1505706360,3772.94,3783.34,3772.91,3783.34,3.777993,14254.511081,3773.037921
4,1505706420,3783.34,3787.10,3783.27,3786.70,0.138679,524.868110,3784.766113
...,...,...,...,...,...,...,...,...
357373,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,NaN,NaN
357374,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,NaN,NaN
357375,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,NaN,NaN
357376,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,NaN,NaN


In [29]:
%%time
#btc_data = pd.read_csv(r"/home/balazs/Desktop/szakdoga_info/btc_data.csv")
#data = btc_data.dropna()
data['MA5']=data['open'].rolling(window =5).mean()
data['EMA5']=data['open'].ewm(span=5,min_periods=0,adjust=False,ignore_na=False).mean()
data['CMA']=data.open.expanding().mean()
data['open_percentage_change']=data.open.pct_change()
data['close_percentage_change']=data.close.pct_change()
data['high_percentage_change']=data.high.pct_change()
data['low_percentage_change']=data.low.pct_change()
data['volume_percentage_change']=data.volume.pct_change()
lista=[]
for i in range(len(data.open_percentage_change.values)-5):
        value=data.open_percentage_change.values[i+1]+data.open_percentage_change.values[i+2]+data.open_percentage_change.values[i+3]+data.open_percentage_change.values[i+4]+data.open_percentage_change.values[i+5]
        lista.append(value)
for i in range(5):
    lista.append(0)
data['profit']=lista
buy_sell_list=[classificateWithNumbersThreeTarget(elem) for elem in data.profit]
data['decision']=buy_sell_list

CPU times: user 48.1 s, sys: 232 ms, total: 48.3 s
Wall time: 49.3 s


In [30]:
data=data.dropna()

In [31]:
data.drop(data.tail(5).index,inplace=True) # drop last n rows

In [32]:
aboveValue(data.open.values,data.MA5.values,data,'aboveMA5')
aboveValue(data.open.values,data.EMA5.values,data,'aboveEMA5')
aboveValue(data.open.values,data.CMA.values,data,'aboveCMA')
data

,time,open,high,low,close,volume,drop1,drop2,MA5,EMA5,...,open_percentage_change,close_percentage_change,high_percentage_change,low_percentage_change,volume_percentage_change,profit,decision,aboveMA5,aboveEMA5,aboveCMA
4,1505706420,3783.34,3787.10,3783.27,3786.70,0.138679,524.868110,3784.766113,3772.036,3773.820494,...,0.002756,0.000888,0.000994,0.002746,-0.963293,-0.000664,0,1,1,1
5,1505706480,3772.99,3783.00,3772.91,3783.00,6.094636,22995.037416,3772.995835,3774.298,3773.543663,...,-0.002736,-0.000977,-0.001083,-0.002738,42.947751,0.002072,1,0,0,1
6,1505706540,3783.00,3785.73,3774.72,3774.72,0.812364,3068.839759,3777.667222,3777.040,3776.695775,...,0.002653,-0.002189,0.000722,0.000480,-0.866708,-0.000127,0,1,1,1
7,1505706600,3785.72,3785.72,3772.94,3784.98,17.533361,66200.068753,3775.663274,3779.598,3779.703850,...,0.000719,0.002718,-0.000003,-0.000472,20.583142,-0.001152,0,1,1,1
8,1505706660,3772.91,3773.02,3772.91,3773.02,0.350105,1320.952047,3773.019171,3779.592,3777.439233,...,-0.003384,-0.003160,-0.003355,-0.000008,-0.980032,0.001562,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499985,1595705280,9696.86,9698.62,9695.95,9695.95,0.210711,2043.143679,9696.447912,9693.810,9693.394325,...,0.000017,-0.000077,-0.000076,-0.000077,-0.682610,-0.000974,0,1,1,1
499986,1595705340,9690.23,9693.70,9684.80,9687.95,0.041349,400.641933,9689.310241,9694.236,9692.339550,...,-0.000684,-0.000825,-0.000507,-0.001150,-0.803765,-0.000290,0,0,0,1
499987,1595705400,9693.64,9693.64,9687.77,9688.41,0.090997,881.978118,9692.427235,9694.458,9692.773033,...,0.000352,0.000047,-0.000006,0.000307,1.200704,-0.000211,0,0,1,1
499988,1595705460,9688.41,9691.34,9683.89,9686.58,0.066272,642.041217,9687.969578,9693.168,9691.318689,...,-0.000540,-0.000189,-0.000237,-0.000401,-0.271709,0.000161,0,0,0,1


In [33]:
X=data[['close_percentage_change','high_percentage_change','low_percentage_change']]
y=data[['decision']]

In [34]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X, y = smote.fit_resample(X, y)

In [35]:
from imblearn.under_sampling import TomekLinks
TK = TomekLinks(sampling_strategy='majority')
X, y = TK.fit_resample(X, y)

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

numeric_pipeline = Pipeline(
    steps=[("impute", SimpleImputer(strategy="mean")), 
           ("scale", StandardScaler())]
)
num_cols = X.select_dtypes(include="number").columns


full_processor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipeline, num_cols)
    ]
)
X_processed = full_processor.fit_transform(X)
y_processed = SimpleImputer(strategy="most_frequent").fit_transform(
    y.values.reshape(-1, 1)
)

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_processed, stratify=y_processed, random_state=1121218
)

In [ ]:
# Init classifier
xgb_cl = xgb.XGBClassifier()

# Fit
xgb_cl.fit(X_train, y_train)

# Predict
preds = xgb_cl.predict(X_test)

# Score
accuracy_score(y_test, preds)

In [14]:
from sklearn.metrics import classification_report
classes = ['-1','0','1']
print(classification_report(y_test, preds, target_names = classes, zero_division = 0))

              precision    recall  f1-score   support

          -1       0.57      0.51      0.54     50602
           0       0.51      0.61      0.56     54775
           1       0.58      0.52      0.55     54776

    accuracy                           0.55    160153
   macro avg       0.55      0.55      0.55    160153
weighted avg       0.55      0.55      0.55    160153



# OPT

In [ ]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [ ]:
space={'max_depth': hp.quniform("max_depth", 2, 7, 1),
        'gamma': hp.uniform ('gamma', 0,4),
       'eta': hp.uniform ('eta', 0.1,0.6),
    }

In [ ]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'],
                    max_depth = int(space['max_depth']),
                    gamma = space['gamma'],
                    eta = space['eta'])
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=3,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        algo = tpe.suggest,
                        max_evals = 50,
                        trials = trials)

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [37]:
# Init classifier
xgb_cl = xgb.XGBClassifier(eta=0.464,gamma=1.67,max_depth=7)

# Fit
xgb_cl.fit(X_train, y_train)

# Predict
preds = xgb_cl.predict(X_test)

# Score
accuracy_score(y_test, preds)

[11:06:23] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.5405654402026833

In [38]:
from sklearn.metrics import classification_report
classes = ['-1','0','1']
print(classification_report(y_test, preds, target_names = classes, zero_division = 0))

              precision    recall  f1-score   support

          -1       0.56      0.47      0.51    240077
           0       0.51      0.64      0.57    262127
           1       0.56      0.50      0.53    261944

    accuracy                           0.54    764148
   macro avg       0.54      0.54      0.54    764148
weighted avg       0.54      0.54      0.54    764148



In [43]:
X_test


,close_percentage_change,high_percentage_change,low_percentage_change
1854911,-0.000518,-0.000001,-0.000216
1787060,-0.004727,-0.007973,-0.005658
1379708,-0.000577,-0.000490,-0.000896
2300086,0.001010,-0.001035,0.001025
383256,-0.000153,0.001089,-0.000153
...,...,...,...
1034812,-0.001492,0.000608,-0.000355
594554,0.001065,0.001065,-0.000016
2435915,0.006373,0.006873,0.002589
2266190,0.002434,0.000766,0.000882


In [44]:
preds

array([ 0, -1,  0, ...,  1,  1,  1])

In [46]:
lista=np.array([[-0.000518,-0.000001,-0.000216]])

xgb_cl.predict(lista)

array([0])

In [47]:
import joblib
joblib.dump(xgb_cl,'xgbBoost.joblib')

['xgbBoost.joblib']